In [43]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
import warnings
import math
import numpy as np
import seaborn as sns
warnings.filterwarnings("ignore")

%matplotlib inline

In [44]:
## Functions

# Funktion, um den Wattwert für einen Prozess aus der 'processes'-Liste zu extrahieren
def extract_wattage(process_list, target_process):
    process_list = ast.literal_eval(process_list)
    for process, wattage in process_list:
        if process == target_process:
            return wattage
    return None

# Funktion um aus allen Messungen (W) den Energie Gesamt Energie-Verbrauch (KwH) für einen Service zu berechnen
def calculate_energy_consumption(process_name,process_wattage_slice,time_delta_slice):
    total_ws = 0.0
    total_t = 0

    for i in range(1,len(process_wattage_slice)):
        dt = time_delta_slice.iloc[i] - time_delta_slice.iloc[i-1]
        average = (process_wattage_slice.iloc[i] + process_wattage_slice.iloc[i-1]) / 2
        total_ws += average * dt
        total_t += dt
    
    if math.isnan(total_ws):
        return 0.0
    
    return total_ws
    

In [47]:
# Load data
from glob import glob
runs = glob(f"data/*/*")

In [46]:
all = None
services = None
max_run = 2
for run in runs:
    exp = run.split('/')[1]
    run_id = int(run.split('/')[2])
    if run_id > max_run:
        continue
    workload = pd.read_csv(f"{run}/locost_stats.csv").tail(1)[["Request Count", "Failure Count","Median Response Time","Average Response Time","Average Content Size"]].rename(
        columns={"Request Count": "requests", "Failure Count": "failures", "Median Response Time": "med. lat.", "Average Response Time": "avg. lat.", "Average Content Size": "avg. size"}
    )
    num_requests = workload["requests"].values[0]
    
    obversations = pd.read_csv(glob(f"{run}/measurements_*.csv")[0])
    obversations["time"] = pd.to_datetime(obversations["timestamp"])
    obversations["processes"] = obversations["processes"].apply(lambda x: ast.literal_eval(x))
    total_runtime = (obversations["time"].max()-obversations["time"].min()).total_seconds()
    X = obversations.explode("processes").dropna()
    X["process_name"] = X["processes"].apply(lambda x: x[0])
    X["wattage"] = X["processes"].apply(lambda x: x[1])
    process_stats = X.groupby("process_name")["process_wattage"].sum().reset_index().rename(columns={"process_wattage": "total_wattage"})
    process_stats["total_wattage"] =process_stats["total_wattage"]/1000000
    process_stats["avg_wattage"] = process_stats["total_wattage"]/total_runtime
    process_stats["avg_wattage_per_request"] = process_stats["total_wattage"]/num_requests

    
    
    run_data = [exp,run_id]+workload.values.tolist()[0]+[total_runtime,process_stats["total_wattage"].sum(),process_stats["process_name"].nunique()]
    run_data = pd.DataFrame([run_data],columns=["exp","run_id","requests","failures","med_lat","avg_lat","avg_payload","runtime","wattage","deployment_units"])

    process_stats["exp"] = exp
    process_stats["run_id"] = run_id
    process_stats["instance"] = process_stats["process_name"].apply(lambda x:X[X["process_name"]==x]["instance"].unique()[0]) 

    if all is None:
        all = run_data
        services = process_stats
    else:
        all = pd.concat([all,run_data])
        services = pd.concat([services,process_stats])
    #     
    #process_measures = X.groupby("process_name").rolling("60s",on="time")["process_wattage"].sum()
    #print(exp,run_id,"\n",workload, "\n",process_stats[["process_name","avg_wattage","avg_wattage_per_request"]], "\ntotal runtime:",total_runtime,"\nreq/s:",workload["requests"].values[0]/total_runtime,"\n\n")
    # plt.figure()
    # sns.lineplot(data=process_measures.reset_index(),x="time",y="process_wattage",hue="process_name")
    # plt.suptitle(f"{exp} - {run_id}")


EmptyDataError: No columns to parse from file

In [ ]:
X = all.groupby(["exp"])[["requests","failures","runtime","wattage","deployment_units"]].sum()
X["mW/Req"] = X["wattage"]/(X["requests"]+X["failures"])*1000
X = X.sort_values(by="mW/Req")
X

,requests,failures,runtime,wattage,deployment_units,mW/Req
exp,,,,,,
jvm_jvm-impoove_cpu,26211.0,26209.0,1093.086901,480.457825,38,9.165544
baseline_vanilla_cpu,3667.0,3600.0,2807.405517,785.064608,56,108.031458
